# Fasttext
Build a fasttext model and do some basic analyses.
Before running this code parse the ogsl data set to produce a dataframe of OGSL sign readings, sign names and utf8 representations, and parse the Ur 3 dataset to produce a text file in cuneiform utf8. 

In [1]:
import pickle
from fastText import train_unsupervised
from fastText import util
import pandas as pd
import numpy as np

In [21]:
model = train_unsupervised('output/ur3.txt', model='skipgram')

In [23]:
labels = model.get_words(include_freq=True)

In [24]:
df = pd.DataFrame({"label": list(labels[0]), "count":list(labels[1])})

In [25]:
df

,count,label
0,130706,𒁹
1,90261,𒋡
2,74950,𒈫
3,68088,𒈬
4,64502,𒐊
5,60049,𒌋
6,57533,xxx
7,56894,𒂆
8,53968,𒐈
9,51460,𒌗


In [10]:
class FastTextNN:
    """by Ali Abul Hawa; see https://github.com/facebookresearch/fastText/pull/552"""    
    def __init__(self, ft_model, ft_matrix=None):
        self.ft_model = ft_model        
        self.ft_words = ft_model.get_words()
        self.word_frequencies = dict(zip(*ft_model.get_words(include_freq=True)))
        self.ft_matrix = ft_matrix
        if self.ft_matrix is None:
            self.ft_matrix = np.empty((len(self.ft_words), ft_model.get_dimension()))
            for i, word in enumerate(self.ft_words):
                self.ft_matrix[i,:] = ft_model.get_word_vector(word)
    
    def find_nearest_neighbor(self, query_word, vectors, n=10,  cossims=None):
        """
        vectors is a 2d numpy array corresponding to the vectors you want to consider

        cossims is a 1d numpy array of size len(vectors), which can be passed for efficiency
        returns the index of the closest n matches to query within vectors and the cosine similarity (cosine the angle between the vectors)

        """
        
        query  = self.ft_model.get_word_vector(query_word)
        if cossims is None:
            cossims = np.matmul(vectors, query, out=cossims)

        norms = np.sqrt((query**2).sum() * (vectors**2).sum(axis=1))
        cossims = cossims/norms
        if query_word in self.ft_words:
            result_i = np.argpartition(-cossims, range(n+1))[1:n+1]
        else:
            result_i = np.argpartition(-cossims, range(n+1))[0:n]
        return list(zip(result_i, cossims[result_i]))

    def nearest_words(self, word, n=10, word_freq=None):
        result = self.find_nearest_neighbor(word, self.ft_matrix, n=n)
        if word_freq:
            return [(self.ft_words[r[0]], r[1]) for r in result if self.word_frequencies[self.ft_words[r[0]]] >= word_freq]
        else:
            return [(self.ft_words[r[0]], r[1]) for r in result]

# Basic usage of nearest_words

In [11]:
fasttext_nn = FastTextNN(model)
fasttext_nn.nearest_words('𒈗', word_freq=150)

[('𒆗𒂵', 0.74875697719869228),
 ('𒌒𒁕', 0.69896564280484219),
 ('𒋀𒀊𒆠𒈠', 0.69174692963293649),
 ('𒀭', 0.6876285945867614)]

# Allow input in transliteration

In [12]:
# create conversion dictionary value to utf8
import pickle
with open("output/ogsl.p", "rb") as p:
    o = pickle.load(p)
d = dict(zip(o["value"], o["utf8"]))

In [13]:
def nearestcun(text, n=10, word_freq=None):
    """input transliteration of a word form, signs separated by hyphens.
    Examples: "ma-an-gi₄", "d-en-lil₂", or "urim₅-ki".
    Transliteration style (gu₄ vs. gud; gen vs. ŋen) is unimportant
    The result is fed to nearest_words.
    n is the number of nearest neigbors that nearest_words will return
    word_freq is the minimum word frequency for a neighbor to be considered"""
    signs = text.lower().split('-')
    seq = [d[s] if s in d else s for s in signs]
    seq = ''.join(seq)
    return fasttext_nn.nearest_words(seq, n, word_freq)

In [37]:
nearestcun("𒅍𒉈", word_freq=10)

[('𒄀𒇡𒇡', 0.7712475476529389),
 ('𒃡𒉈', 0.76110806625205829),
 ('𒄀𒄥𒅘', 0.75588822490246932),
 ('𒅍𒅍𒉈', 0.75576223066764947),
 ('𒋢𒃻', 0.74806277480867089)]

# TODO
Transliterate output by finding the word in the corpus

In [40]:
model.get_subwords("𒄀𒇡𒇡")

(['𒄀𒇡𒇡', '<𒄀𒇡', '<𒄀𒇡𒇡', '<𒄀𒇡𒇡>', '𒄀𒇡𒇡', '𒄀𒇡𒇡>', '𒇡𒇡>'],
 array([   5866,  262905, 1923231, 1288427,  500893, 1315217,  998123]))

In [48]:
model.save_model("output/ur3model")